In [ ]:
from typing import Tuple
import numpy as np
import numba as nb
from scipy import stats
from collections import namedtuple
from matplotlib import pyplot as plt
import sys
from matplotlib.pyplot import figure as fig
import time
import math


"""
This is a simple container for the properties of the inventory at a given
timestep.
"""
State = namedtuple(
    "State",
    [
        "inventory",
        "in_transit",
        "stockout",
        "satisfied_demand",
        "unsatisfied_demand",
        "replenishment",
    ]
)

In [ ]:
%matplotlib inline

In [ ]:
@nb.jit(nopython=True, fastmath=True, nogil=True)
def process_demand(
    inventory: np.int64, 
    demand: np.int64
) -> Tuple[np.int64, np.int64, np.int64]:
    """ Process demand on a given timestep.

    This is a very simple function used to isolate this section of simulation
    logic. We can add more complicated behaviour in the future.

    Parameters
    ----------
    inventory: int
        The inventory on hand at this timestep.
    demand: int
        The demand on this timestep.

    Returns
    -------
    new_inventory: int
        The inventory on hand after processing the demand on this timestep.
    unsatisfied_demand: int
        The unsatisfied demand on this timestep, i.e., the demand which could
        not be served from on-hand inventory.

    Notes
    -----
    We can design another function to handle unsatisfied_demand in different
    ways, for example by computing backorders and adding those to the demand
    timeseries.
    """
    satisfied_demand = min(inventory, demand)
    new_inventory = max(inventory - demand, 0)
    unsatisfied_demand = demand - satisfied_demand
    return new_inventory, satisfied_demand, unsatisfied_demand


@nb.jit(nopython=True, fastmath=True, nogil=True)
def r_s_policy(
    t: np.int64,
    state: State,
    demand_ts: np.ndarray,
    replen_ts: np.ndarray,
    review_period: np.int64,
    up_to_level: np.int64,
    lead_times_dist: np.ndarray,
    num_timesteps: np.int64,
    backorders: bool,
) -> Tuple[State, np.ndarray, np.ndarray]:
    """ Simulate a single timestep using a simple R-S inventory policy.

    This is a fixed review period, fixed order-up-to-quantity policy.
    
    1. Check if the demand on this timestep is less than the current 
        inventory.
        1a. If the demand is less than the current inventory there is no
            stockout.
        1b. If the demand is greater than the current inventory there is a
            stockout.
    2. Record any unsatisfied demand:
        2a. If processing backorders, add any unsatisfied from this timestep
            onto the demand for timestep t+1.
    3. If in the review period:
        3a. Check the current net inventory by summing the current on hand 
            inventory and the in transit inventory.
        3b. Calculate the difference between the net inventory and the
            up-to-level (S) as a "new order".
        3c. Update the record of what is currently in transit, and sample the
            lead time for the new order.
        3d. If the order is due to arrive before the simulation finishes, add
            it to the replenishment timeseries (replen_ts).
    4. Receive new orders:
        4a. Add received orders on this timestep to the current inventory
        4b. Subtract received orders on this timestep from in transit

    Parameters
    ----------
    t: int
        The current timestep (starting from 0).
    state: State
        Namedtuple containing:
        - inventory: int
            The on hand inventory at the start of this timestep
        - in_transit: int
            The quantity of in transit items at the start of this timestep
        - stockout: bool
            Whether a stockout occured in the previous timestep.
    demand_ts: np.ndarray
        Length (num_timesteps) array of integers describing the demand at each
        timestep.
    replen_ts: np.ndarray
        Length (num_timesteps) array of integers describing the replenishment 
        that occurs on timestep. The entry of replen_ts at position t is added
        to the on hand inventory at the end of timestep t. This array is
        mutated by this function when new orders are made.
    review_period: int
        Mandatory keyword argument describing the review period. A review
        period of 1 means to review every timestep.
    up_to_level: int
        Mandatory keyword argument describing the up_to_level ('S').
    lead_times_dist: np.ndarray
        1D array of integers describing possible lead times. The lead
        time for a new order is sampled from this array. If the array has 
        size 1, the lead time is deterministic (as the same lead time will be 
        returned every time). For stochastic lead times, pre-compute a large
        sample of candidate lead times and pass them to this function.
    num_timesteps: int
        Mandatory keyword argument describing the number of timesteps in the
        simulation. Currently only used to prevent out of bounds access on 
        replen_ts.
    backorders: bool
        Flag indicating whether backorders should be present (True) or not
        (False). Backorders add any unsatisfied demand from the current
        timestep, t, onto the demand at timestep t + 1.

    Returns
    -------
    new_state: State
        Namedtuple containing:
        - inventory: int
            The on hand inventory at the end of this timestep
        - in_transit: int
            The quantity of in transit items at the end of this timestep
        - stockout: bool
            Whether a stockout occured in this timestep.
    demand_ts: np.ndarray
        Length (num_timesteps) array of integers describing the demand at each
        timestep. This can feasibly be modified by this function if
        backorders should be added to future demand, for example.
    replen_ts: np.ndarray
        Length (num_timesteps) array of integers describing the replenishment 
        that occurs on timestep. The entry of replen_ts at position t is added
        to the on hand inventory at the end of timestep t. This array is
        mutated by this function when new orders are made.
    """

    # demand & replenishment on this timestep
    demand = demand_ts[t]
    replen = replen_ts[t]

    # extract relevant state variables
    inventory = state.inventory
    in_transit = state.in_transit
    stockout = np.int64(demand >= inventory)

    (
        inventory, satisfied_demand, unsatisfied_demand
    ) = process_demand(inventory, demand)

    # process any backorders
    if backorders:
        if stockout == 1 and (t+1) < num_timesteps:
            demand_ts[t + 1] += unsatisfied_demand

    # make new orders if we're in the review period
    if t % review_period == 0:

        net_inventory = inventory + in_transit
        raw_order_qty = max(up_to_level - net_inventory, 0)

        # logic for manipulating order qty, e.g. due to pack size, should go here
        order_qty = raw_order_qty

        # update what is in transit
        in_transit = in_transit + order_qty

        # order arrives at lead_time timesteps from the current timestep
        lead_time = np.random.choice(lead_times_dist)
        order_arrives = t + lead_time

        # if it arrives before the simulation finishes, update
        # the orders timeseries
        if order_arrives < num_timesteps:

            replen_ts[order_arrives] += order_qty

    in_transit = in_transit - replen
    inventory = inventory + replen

    new_state = State(
        inventory=inventory,
        in_transit=in_transit,
        stockout=stockout,
        satisfied_demand=satisfied_demand,
        unsatisfied_demand=unsatisfied_demand,
        replenishment=replen
    )

    return new_state, demand_ts, replen_ts

In [ ]:
@nb.jit(nopython=True, fastmath=True, nogil=True)
def run_simulation_internal(
    demand_ts: np.ndarray,
    review_period: np.int64,
    nominal_lead_time: np.int64,
    std_lead_time: np.int64,
    lead_times_dist: np.int64,
    backorders: bool,
    up_to_level: np.int64,
    initial_inventory: np.int64,
    initial_in_transit: np.int64,
    initial_stockout: np.int64,
    initial_satisfied_demand: np.int64,
    initial_unsatisfied_demand: np.int64,
    initial_replenishment: np.int64,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """ Run a single simulation.

    This function handles proper initialisation of all the required
    variables and results containers, and is responsible for stepping through
    the required number of timesteps (determined by demand_ts).

    Parameters
    ----------
    demand_ts: np.ndarray
        Size (num_timesteps) array of integers containing the demand at each
        timestep.
    review_period: int
        The review period. A review period of 1 means to review every
        timestep.
    nominal_lead_time: int
        The best guess / expectedlead time. Not necessarily the
        mean because orders tend to be late rather than early.
    lead_times_dist: np.ndarray
        1D array of integers describing possible lead times. The lead
        time for a new order is sampled from this array. If the array has
        size 1, the lead time is deterministic (as the same lead time will be
        returned every time). For stochastic lead times, pre-compute a large
        sample of candidate lead times and pass them to this function.
    backorders: bool
        Flag indicating whether backorders should be present (True) or not
        (False). Backorders add any unsatisfied demand from the current
        timestep, t, onto the demand at timestep t + 1.
    up_to_level: int
        The up_to_level ('S').
    initial_inventory: int
        The amount of (on hand) inventory at timestep t=0.
    initial_in_transit: int
        The amount of inventory currently in transit (i.e. on order) at 
        timestep t=0. Used to populate the initial replenishment timeseries.
    initial_stockout: int
        Whether a stockout occured at timestep t=0. Currently doesn't do 
        anything.
    initial_satisfied_demand: int
        The amount of satisfied demand (demand serviced from on hand 
        inventory) at timestep t=0.
    initial_unsatisfied_demand: int
        The amount of unsatisfied demand (demand in excess of the on hand 
        inventory) at timestep t=0.
    initial_replenishment: int
        The amount of replenishment at timestep t=0. This doesn't do anything,
        since we compute the initial inventory without it, but it is here for
        consistency / to make the trace timeseries look more pretty at t=0.

    Returns
    -------
    trace_inventory: np.ndarray
        Size (num_timesteps) array containing the on hand inventory at the
        end (after replenishment) of each timestep.
    trace_in_transit: np.ndarray
        Size (num_timesteps) array containing the quantity of in transit
        (i.e. "on order") items at the end of each timestep.
    trace_stockout: np.ndarray
        Size (num_timesteps) array indicating whether a stockout occured on 
        that timestep.
    trace_satisfied: np.ndarray
        Size (num_timesteps) array containing the satisfied demand (demand 
        serviced from on hand inventory) on each timestep.
    trace_unsatisfied: np.ndarray
        Size (num_timesteps) array containing the unsatisfied demand (demand 
        in excess of the on hand inventory) on each timestep.
    trace_replenishment: np.ndarray
        Size (num_timesteps) array containing the replenishment at each 
        timestep.
    """

    # we can infer the number of timesteps from the demand timeseries
    num_timesteps = demand_ts.shape[0]

    # initialise simulator state
    state = State(
        inventory=initial_inventory,
        in_transit=initial_in_transit,
        stockout=initial_stockout,
        satisfied_demand=initial_satisfied_demand,
        unsatisfied_demand=initial_unsatisfied_demand,
        replenishment=initial_replenishment
    )

    # initialise the replen timeseries
    replen_ts = np.zeros(num_timesteps, dtype=np.int64)
    replen_ts[nominal_lead_time] = state.in_transit

    # containers
    trace_inventory = np.empty(num_timesteps, dtype=np.int64)
    trace_in_transit = np.empty(num_timesteps, dtype=np.int64)
    trace_stockout = np.empty(num_timesteps, dtype=np.int64)
    trace_satisfied = np.empty(num_timesteps, dtype=np.int64)
    trace_unsatisfied = np.empty(num_timesteps, dtype=np.int64)
    trace_replenishment = np.empty(num_timesteps, dtype=np.int64)

    # first timestep
    trace_inventory[0] = state.inventory
    trace_in_transit[0] = state.in_transit
    trace_stockout[0] = state.stockout
    trace_satisfied[0] = state.satisfied_demand
    trace_unsatisfied[0] = state.unsatisfied_demand
    trace_replenishment[0] = state.replenishment

    # iterate over t > 0
    for t in range(1, num_timesteps):
        state, demand_ts, replen_ts = r_s_policy(
            t,
            state,
            demand_ts,
            replen_ts,
            review_period,
            up_to_level,
            lead_times_dist,
            num_timesteps,
            backorders
        )

        # store results after each iteration
        trace_inventory[t] = state.inventory
        trace_in_transit[t] = state.in_transit
        trace_stockout[t] = state.stockout
        trace_satisfied[t] = state.satisfied_demand
        trace_unsatisfied[t] = state.unsatisfied_demand
        trace_replenishment[t] = state.replenishment

    return (
        trace_inventory,
        trace_in_transit,
        trace_stockout,
        trace_satisfied,
        trace_unsatisfied,
        trace_replenishment
    )

In [ ]:
@nb.jit(nopython=True, nogil=True, fastmath=True)
def run_simulations(
    demand_array: np.ndarray,
    review_period: np.int64,
    nominal_lead_time: np.int64,
    std_lead_time: np.int64,
    lead_times_dist: np.ndarray,
    backorders: bool,
    up_to_levels: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """ Essentially a jitted wrapper around run_simulation_internal.

    Executes multiple simulations in parallel once all the necessary 
    variables have been pre-computed.
    """

    num_simulations, num_timesteps = demand_array.shape
    output_size = (num_simulations, num_timesteps)

    # containers
    trace_inventory = np.empty(output_size, dtype=np.int64)
    trace_in_transit = np.empty(output_size, dtype=np.int64)
    trace_stockout = np.empty(output_size, dtype=np.int64)
    trace_satisfied = np.empty(output_size, dtype=np.int64)
    trace_unsatisfied = np.empty(output_size, dtype=np.int64)
    trace_replenishment = np.empty(output_size, dtype=np.int64)

    for i in nb.prange(0, num_simulations):

        up_to_level = up_to_levels[i]

        demand_ts = demand_array[i, :].copy()

        initial_inventory = up_to_level - demand_ts[0]
        initial_in_transit = demand_ts[0]
        initial_stockout = 0
        initial_satisfied_demand = demand_ts[0]
        initial_unsatisfied_demand = 0
        initial_replenishment = 0

        (
            trace_inventory[i, :],
            trace_in_transit[i, :],
            trace_stockout[i, :],
            trace_satisfied[i, :],
            trace_unsatisfied[i, :],
            trace_replenishment[i, :]
        ) = run_simulation_internal(
            demand_ts,
            review_period,
            nominal_lead_time,
            std_lead_time,
            lead_times_dist,
            backorders,
            up_to_level,
            initial_inventory,
            initial_in_transit,
            initial_stockout,
            initial_satisfied_demand,
            initial_unsatisfied_demand,
            initial_replenishment,
        )

    return (
        trace_inventory,
        trace_in_transit,
        trace_stockout,
        trace_satisfied,
        trace_unsatisfied,
        trace_replenishment
    )

In [ ]:
def normal_volatility(
    demand_train: np.ndarray,
    alpha: np.float64,
    nominal_lead_time: np.int64,
    std_lead_time: np.int64,
    lead_times_dist: np.ndarray,
    review_period: np.int64
) -> np.int64:
    demand_mean = np.mean(demand_train)
    demand_std = np.std(demand_train, ddof=1)
    
    z = stats.norm.ppf(alpha)
    x_std = np.sqrt(((nominal_lead_time + review_period)* np.square(demand_std)) + np.square(std_lead_time)* 
                    np.square(demand_mean))
    # calculate the up-to-level (assumes demand is Normally distributed):
    # up-to-level is the sum of the cycle stock, the average in transit, and
    # the safety stock
    safety_stock = x_std * z
    
    cycle_stock = demand_mean * review_period
    avg_in_transit = demand_mean * nominal_lead_time
    up_to_level = np.round(safety_stock + cycle_stock + avg_in_transit)
    return np.int64(up_to_level)


def empirical_volatility(
    demand_train: np.ndarray,
    alpha: np.float64,
    nominal_lead_time: np.int64,
    std_lead_time: np.int64,
    lead_times_dist: np.ndarray,
    review_period: np.int64
) -> np.int64:
    agg_level = nominal_lead_time + review_period
    cum_sum = np.cumsum(demand_train)
    rolling_sum = np.empty(cum_sum.shape[0] - agg_level)
    rolling_sum = cum_sum[agg_level:] - cum_sum[:-agg_level]
    up_to_level = np.quantile(rolling_sum, alpha)
    return np.int64(up_to_level)



In [ ]:
VOLATILITY_MODELS = {
    'normal': normal_volatility,
    'empirical': empirical_volatility
}


def run_experiment(
    *,
    demand_array: np.ndarray,
    nominal_lead_time: np.int64,
    std_lead_time: np.int64,
    lead_times_dist: np.ndarray,
    review_period: np.int64,
    alpha: np.float64,
    backorders: bool,
    num_train: np.int64,
    volatility = 'normal'
):

    num_simulations, total_timesteps = demand_array.shape

    # first precompute the up to level for each simulation using the desired
    # number of training samples
    volatility_model = VOLATILITY_MODELS[volatility]

    up_to_levels = np.empty(num_simulations, dtype=np.int64)
    for i in range(num_simulations):
        up_to_levels[i] = volatility_model(
            demand_array[i, :num_train],
            alpha,
            nominal_lead_time,
            std_lead_time,
            lead_times_dist,
            review_period,
        )

    # only pass in the timesteps after the training period
    (
        trace_inventory,
        trace_in_transit,
        trace_stockout,
        trace_satisfied,
        trace_unsatisfied,
        trace_replenishment
    ) = run_simulations(
            demand_array[:, num_train:],
            review_period,
            nominal_lead_time,
            std_lead_time,
            lead_times_dist,
            backorders,
            up_to_levels
    )

    # remove the warmup / initialisation period
    cutoff = nominal_lead_time + review_period

    trace_inventory = trace_inventory[:, cutoff:]
    trace_in_transit = trace_in_transit[:, cutoff:]
    trace_stockout = trace_stockout[:, cutoff:]
    trace_satisfied = trace_satisfied[:, cutoff:]
    trace_unsatisfied = trace_unsatisfied[:, cutoff:]
    trace_replenishment = trace_replenishment[:, cutoff:]
    demand_for_ratio = demand_array[:, num_train + cutoff:]
    
    inventory_demand_ratio = (trace_inventory.mean(axis=1)/ demand_for_ratio.mean(axis=1))
    period_service_levels = 1 - (trace_stockout.sum(axis=1) / trace_stockout.shape[1])
    cycle_service_levels = calculate_cycle_service_levels(trace_stockout, review_period)
    fill_rates = 1 - (trace_unsatisfied.sum(axis=1) / (trace_satisfied.sum(axis=1) + trace_unsatisfied.sum(axis=1)))

    return period_service_levels, cycle_service_levels, fill_rates, inventory_demand_ratio


def calculate_cycle_service_levels(
    trace_stockout: np.ndarray,
    review_period: np.int64
) -> np.ndarray:
    """ Calculate cycle service level given the trace of period service
    level stockouts (trace_stockout), where the trace_stockout array is 
    assumed to be of shape  (num_simulations, num_timesteps).

    The cycle duration is presumed to be the number of timesteps in the review
    period.

    The final cycle may include fewer than the complete number of timesteps if
    num_timesteps is not divisible by review_period without remainder.

    Parameters
    ----------
    trace_stockout: np.ndarray
        Size (num_simulations, num_timesteps) array containing 1 when a 
        stockout occured on that period (i.e. whether a period stockout 
        ocurred or not), and 0 otherwise.
    review_period: int
        Integer describing the length of the review period (equivalently, the
        duration of a stocking cycle).

    Returns
    -------
    cycle_service_levels: np.ndarray
        Size (num_simulations) array containing the cycle service level for
        each simulation described in trace_stockout. The cycle service level
        describes whether a stockout occured during that order cycle, presumed
        to be of duration equivalent to review_period.
    """

    num_simulations, num_timesteps = trace_stockout.shape

    # work out the number of order cycles
    n_cycles = num_timesteps // review_period

    # create a container for the results. Each simulation contains
    # n_cycles order cycles
    stockout_cycle = np.zeros((num_simulations, n_cycles), dtype=np.int64)

    # split the period stockouts into cycles
    cycles = np.array_split(trace_stockout, n_cycles, axis=1)

    # iterate over the cycles
    for i, cycle in enumerate(cycles):
        # record the cycles where a stockout occurred
        stockout_cycle[:, i] = np.any(cycle, axis=1)

    # cycle service level is 1 - fraction of cycle stockouts
    cycle_service_levels = (
        1 - stockout_cycle.sum(axis=1) / stockout_cycle.shape[1]
    )
    return cycle_service_levels

In [ ]:
def generate_normal_demand(
    *,
    demand_mean,
    demand_sd,
    num_simulations,
    num_timesteps_train,
    num_timesteps_test
):
    total_timesteps = num_timesteps_train + num_timesteps_test
    raw = np.random.normal(
        demand_mean,
        demand_sd,
        size=(num_simulations, total_timesteps)
    )
    return np.round(raw).astype(np.int64)


def generate_poisson_demand(
    *,
    demand_rate,
    num_simulations,
    num_timesteps_train,
    num_timesteps_test
):
    total_timesteps = num_timesteps_train + num_timesteps_test
    raw = np.random.poisson(
        demand_rate,
        size=(num_simulations, total_timesteps)
    )
    return np.round(raw).astype(np.int64)


def generate_zero_inflated_poisson_demand(
    *,
    demand_rate,
    p_demand,
    num_simulations,
    num_timesteps_train,
    num_timesteps_test
):
    total_timesteps = num_timesteps_train + num_timesteps_test
    raw = generate_zero_inflated_poisson_demand_internal(
        demand_rate,
        p_demand,
        num_simulations,
        total_timesteps
    )
    return np.round(raw).astype(np.int64) 


@nb.jit(nopython=True)
def generate_zero_inflated_poisson_demand_internal(
    *,
    demand_rate,
    p_demand,
    num_simulations,
    total_timesteps
):
    total_timesteps = num_timesteps_train + num_timesteps_test
    raw = np.zeros((num_simulations, total_timesteps), dtype=np.int64)
    for i in range(num_simulations):
        raw[i, :] = np.random.binomial(1, p_demand, size=total_timesteps)
        for j in range(total_timesteps):
            if raw[i, j] == 1:
                raw[i, j] = np.random.poisson(demand_rate)
    return raw

In [ ]:
from collections import ChainMap

DEMAND_MODELS = {
    "normal": generate_normal_demand,
    "poisson": generate_poisson_demand,
    "zip": generate_zero_inflated_poisson_demand,
}

def run_study(
    *,
    num_simulations,
    num_timesteps_train,
    num_timesteps_test,
    demand_model,
    demand_model_params,
    volatility,
    nominal_lead_time,
    std_lead_time,
    lead_times_dist,
    review_period,
    alpha,
    backorders,
):
    demand_params_base = {
        "num_simulations": num_simulations,
        "num_timesteps_train": num_timesteps_train,
        "num_timesteps_test": num_timesteps_test
    }
    
    demand_model = DEMAND_MODELS[demand_model]
    demand_params = ChainMap(demand_model_params, demand_params_base)
    demand_array = demand_model(**demand_params)
    
    period_service_levels, cycle_service_levels, fill_rates, inventory_demand_ratio = run_experiment(
        demand_array=demand_array,
        nominal_lead_time=nominal_lead_time,
        std_lead_time=std_lead_time,
        lead_times_dist=lead_times_dist,
        review_period=review_period,
        alpha=alpha,
        backorders=backorders,
        num_train=num_train,
        volatility=volatility,
    )

    return period_service_levels, cycle_service_levels, fill_rates, inventory_demand_ratio

In [ ]:
# Plotting function for achieved service level
def plotting_fucn_sl(title):
    fig, ax = plt.subplots(figsize=(12, 8))

    plt.plot(alpha_array, mean_alpha_csl , label='Average Cycle Service Levels')
    plt.fill_between(alpha_array, max_alpha_csl, min_alpha_csl, alpha=.5, linewidth=0)

    plt.plot(alpha_array, mean_alpha_p , label='Average Period Service Level')
    plt.fill_between(alpha_array, max_alpha_p, min_alpha_p, alpha=.5, linewidth=0)


    plt.plot(alpha_array, mean_beta, label='Average Fill Rates')
    plt.fill_between(alpha_array, max_beta, min_beta, alpha=.5, linewidth=0)


    plt.grid(axis='x', color='0.95')
    
    ax.set(xlabel='Target Service Level', ylabel='Achieved Service Level') 
    ax.set_title(title)
    plt.legend(loc='lower right', prop={'size': 12})
    plt.xlim(0.7 , 0.99)
    plt.show()
#     filename =  'sl' +'.png'
#     plt.savefig(filename, bbox_inches='tight')
    
    return (ax)

In [ ]:
# Plotting function for lead time
def plotting_fucn_lt(title):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    plt.plot(alpha_array, csl_for_lead_time[0,:], label = 'Lead Time 2')
    
    plt.plot(alpha_array, csl_for_lead_time[1,:], label = 'Lead Time 4')
    
    plt.plot(alpha_array, csl_for_lead_time[2,:], label = 'Lead Time 10')

    plt.grid(axis='x', color='0.95')
    
    ax.set(xlabel='Target Service Level', ylabel='Achieved Service Level') 
    ax.set_title(title)
    plt.legend(loc='lower right', prop={'size': 12})
    plt.xlim(0.7 , 0.99)
    plt.show()
#     filename =  str(title) +'.png'
#     plt.savefig(filename, bbox_inches='tight')
    
    return (ax)

In [ ]:
# Plotting function for lead time
def plotting_fucn_ratio(title):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    plt.plot(alpha_array, mean_inv_dem_ratio[0,:],  label = 'Normal Volatility')
    plt.plot(alpha_array, mean_inv_dem_ratio[1,:],  label = 'Empirical Volatility')


    plt.grid(axis='x', color='0.95')
    
    ax.set(xlabel='Target Service Level', ylabel='Inventory-Demand Ratio') 
    ax.set_title(title)
    plt.legend(loc='lower right', prop={'size': 12})
    plt.xlim(0.7 , 0.99)
    plt.show()
#     filename = str(title) +'.png'
#     plt.savefig(filename, bbox_inches='tight')
    
    return (ax)

In [ ]:
# Plotting function for lead time
def plotting_fucn_ratio_CSL(title):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    plt.plot(mean_alpha_csl_n, mean_inv_dem_ratio[0,:],  label = 'Normal Volatility')
    plt.plot(mean_alpha_csl, mean_inv_dem_ratio[1,:],  label = 'Empirical Volatility')


    plt.grid(axis='x', color='0.95')
    
    ax.set(xlabel='Achieved Service Level', ylabel='Inventory-Demand Ratio') 
    ax.set_title(title)
    plt.legend(loc='lower right', prop={'size': 12})
#     plt.xlim(0.7 , 0.99)
    plt.show()
#     filename =  str(title) +'.png'
#     plt.savefig(filename, bbox_inches='tight')
    
    return (ax)

In [ ]:
#### Demand: Normal
##### Lead Time: Stochastic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for l in lead_time_array:
    for b in backorder_array:
        mean_inv_dem_ratio = np.empty((0,59), float)
        for v in volatility_array:
                alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
                alpha_p = np.empty((0,1000), float)
                alpha_csl = np.empty((0,1000), float)
                beta = np.empty((0,1000), float)
                inv_dem_ratio = np.empty((0,1000), float)
                title = 'Demand: Normal', 'Backorder: ',b,'Volatility',v,'Stochastic Lead Time: ',l
                for a in alpha_array:
                    demand_model ='normal'
                    nominal_lead_time = np.int64(l)
                    std_lead_time = 2
                    lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                    volatility = v
                    backorders = b
                    num_simulations = 1000
                    num_timesteps_train = 1000
                    num_timesteps_test = 100
                    demand_mean = 100
                    demand_sd = 20
                    review_period = np.int64(4)
                    alpha = a
                    num_train = 365
                    demand_model_params = {"demand_mean": 100, "demand_sd": 10}

                    period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                    num_simulations=num_simulations,
                    num_timesteps_train=num_timesteps_train,
                    num_timesteps_test=num_timesteps_test,
                    demand_model=demand_model,
                    demand_model_params=demand_model_params,
                    volatility=volatility,
                    nominal_lead_time=nominal_lead_time,
                    std_lead_time=std_lead_time,
                    lead_times_dist=lead_times_dist,
                    review_period=review_period,
                    alpha=alpha,
                    backorders=backorders,
                    )

                    alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                    alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                    beta = np.append(beta, np.array([fill_rates]), axis=0)
                    inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                    
#                     aa = round(alpha, 2)
#                     if aa == 0.80 or aa == 0.85 or aa == 0.90 or aa == 0.95 or aa == 0.99:
#                         print(aa)
#                         print(alpha)
#                         print(round(period_service_levels.mean(),3))
#                         print(round(cycle_service_levels.mean(),3))
#                         print(round(fill_rates.mean(),3))

                min_alpha_p = alpha_p.min(axis=1)
                min_alpha_csl = alpha_csl.min(axis=1)
                min_beta = beta.min(axis=1)
                max_alpha_p = alpha_p.max(axis=1)
                max_alpha_csl = alpha_csl.max(axis=1)
                max_beta = beta.max(axis=1)
                mean_alpha_p = alpha_p.mean(axis=1)
                mean_alpha_csl = alpha_csl.mean(axis=1)
                mean_beta = beta.mean(axis=1)
                
                if v == 'normal':
                    mean_alpha_csl_n = mean_alpha_csl
                
                mean_inv_dem_ratio = np.append(mean_inv_dem_ratio, np.array([ inv_dem_ratio.mean(axis=1)]), axis=0)
               

                print('Lead Time: ',l)
                print('Volatility: ',v)
                print('Backorders: ',b)                
                plotting_fucn_sl(title)
        if v == 'empirical':
            plotting_fucn_ratio(title)
            
            plotting_fucn_ratio_CSL(title)
                
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Demand: Normal
##### Lead Time: Deterministic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for l in lead_time_array:
    for b in backorder_array:
        mean_inv_dem_ratio = np.empty((0,59), float)
        for v in volatility_array:
                alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
                alpha_p = np.empty((0,1000), float)
                alpha_csl = np.empty((0,1000), float)
                beta = np.empty((0,1000), float)
                inv_dem_ratio = np.empty((0,1000), float)
                title = 'Demand: Normal', 'Backorder: ',b,'Volatility',v,'Deterministic Lead Time: ',l
                for a in alpha_array:
                    demand_model ='normal'
                    nominal_lead_time = np.int64(l)
                    std_lead_time = 0
                    lead_times_dist = np.array([l], dtype=np.int64)
                    volatility = v
                    backorders = b
                    num_simulations = 1000
                    num_timesteps_train = 1000
                    num_timesteps_test = 100
                    demand_mean = 100
                    demand_sd = 20
                    review_period = np.int64(4)
                    alpha = a
                    num_train = 365
                    demand_model_params = {"demand_mean": 100, "demand_sd": 10}

                    period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                    num_simulations=num_simulations,
                    num_timesteps_train=num_timesteps_train,
                    num_timesteps_test=num_timesteps_test,
                    demand_model=demand_model,
                    demand_model_params=demand_model_params,
                    volatility=volatility,
                    nominal_lead_time=nominal_lead_time,
                    std_lead_time=std_lead_time,
                    lead_times_dist=lead_times_dist,
                    review_period=review_period,
                    alpha=alpha,
                    backorders=backorders,
                    )

                    alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                    alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                    beta = np.append(beta, np.array([fill_rates]), axis=0)
                    inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                    
#                     aa = round(alpha, 2)
#                     if aa == 0.80 or aa == 0.85 or aa == 0.90 or aa == 0.95 or aa == 0.99:
#                         print(aa)
#                         print(alpha)
#                         print(round(period_service_levels.mean(),3))
#                         print(round(cycle_service_levels.mean(),3))
#                         print(round(fill_rates.mean(),3))
                        
                min_alpha_p = alpha_p.min(axis=1)
                min_alpha_csl = alpha_csl.min(axis=1)
                min_beta = beta.min(axis=1)
                max_alpha_p = alpha_p.max(axis=1)
                max_alpha_csl = alpha_csl.max(axis=1)
                max_beta = beta.max(axis=1)
                mean_alpha_p = alpha_p.mean(axis=1)
                mean_alpha_csl = alpha_csl.mean(axis=1)
                mean_beta = beta.mean(axis=1)
                
                if v == 'normal':
                    mean_alpha_csl_n = mean_alpha_csl
                    
                mean_inv_dem_ratio = np.append(mean_inv_dem_ratio, np.array([ inv_dem_ratio.mean(axis=1)]), axis=0)
               
                
                    
                print('Lead Time: ',l)
                print('Volatility: ',v)
                print('Backorders: ',b)                
                plotting_fucn_sl(title)
        if v == 'empirical':
            plotting_fucn_ratio(title)
            plotting_fucn_ratio_CSL(title)
            
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Demand: Poisson
##### Lead Time: Deterministic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for l in lead_time_array:
    for b in backorder_array:
        mean_inv_dem_ratio = np.empty((0,59), float)
        for v in volatility_array:
                alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
                alpha_p = np.empty((0,1000), float)
                alpha_csl = np.empty((0,1000), float)
                beta = np.empty((0,1000), float)
                inv_dem_ratio = np.empty((0,1000), float)
                title = 'Demand: Poisson', 'Backorder: ',b,'Volatility',v,'Deterministic Lead Time: ',l
                for a in alpha_array:
                    demand_model ='poisson'
                    nominal_lead_time = np.int64(l)
                    std_lead_time = 0
                    lead_times_dist = np.array([l], dtype=np.int64)
                    volatility = v
                    backorders = b
                    num_simulations = 1000
                    num_timesteps_train = 1000
                    num_timesteps_test = 100
                    demand_rate = 100
#                     demand_mean = 100
#                     demand_sd = 20
                    review_period = np.int64(4)
                    alpha = a
                    num_train = 365
                    demand_model_params = {"demand_rate": 100}

                    period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                    num_simulations=num_simulations,
                    num_timesteps_train=num_timesteps_train,
                    num_timesteps_test=num_timesteps_test,
                    demand_model=demand_model,
                    demand_model_params=demand_model_params,
                    volatility=volatility,
                    nominal_lead_time=nominal_lead_time,
                    std_lead_time=std_lead_time,
                    lead_times_dist=lead_times_dist,
                    review_period=review_period,
                    alpha=alpha,
                    backorders=backorders,
                    )

                    alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                    alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                    beta = np.append(beta, np.array([fill_rates]), axis=0)
                    inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                    
#                     aa = round(alpha, 2)
#                     if aa == 0.80 or aa == 0.85 or aa == 0.90 or aa == 0.95 or aa == 0.99:
#                         print(aa)
#                         print(alpha)
#                         print(round(period_service_levels.mean(),3))
#                         print(round(cycle_service_levels.mean(),3))
#                         print(round(fill_rates.mean(),3))
                        
                min_alpha_p = alpha_p.min(axis=1)
                min_alpha_csl = alpha_csl.min(axis=1)
                min_beta = beta.min(axis=1)
                max_alpha_p = alpha_p.max(axis=1)
                max_alpha_csl = alpha_csl.max(axis=1)
                max_beta = beta.max(axis=1)
                mean_alpha_p = alpha_p.mean(axis=1)
                mean_alpha_csl = alpha_csl.mean(axis=1)
                mean_beta = beta.mean(axis=1)
                
                if v == 'normal':
                    mean_alpha_csl_n = mean_alpha_csl
                
                mean_inv_dem_ratio = np.append(mean_inv_dem_ratio, np.array([ inv_dem_ratio.mean(axis=1)]), axis=0)
               

                print('Lead Time: ',l)
                print('Volatility: ',v)
                print('Backorders: ',b)                
                plotting_fucn_sl(title)
        if v == 'empirical':
            plotting_fucn_ratio(title)
            plotting_fucn_ratio_CSL(title)
                
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Demand: Poisson
##### Lead Time: Stochastic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for l in lead_time_array:
    for b in backorder_array:
        mean_inv_dem_ratio = np.empty((0,59), float)
        for v in volatility_array:
                alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
                alpha_p = np.empty((0,1000), float)
                alpha_csl = np.empty((0,1000), float)
                beta = np.empty((0,1000), float)
                inv_dem_ratio = np.empty((0,1000), float)
                title = 'Demand: Poisson', 'Backorder: ',b,'Volatility',v,'Stochastic Lead Time: ',l
                for a in alpha_array:
                    demand_model ='poisson'
                    nominal_lead_time = np.int64(l)
                    std_lead_time = 2
                    lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                    volatility = v
                    backorders = b
                    num_simulations = 1000
                    num_timesteps_train = 1000
                    num_timesteps_test = 100
                    demand_rate = 100
#                     demand_mean = 100
#                     demand_sd = 20
                    review_period = np.int64(4)
                    alpha = a
                    num_train = 365
                    demand_model_params = {"demand_rate": 100}

                    period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                    num_simulations=num_simulations,
                    num_timesteps_train=num_timesteps_train,
                    num_timesteps_test=num_timesteps_test,
                    demand_model=demand_model,
                    demand_model_params=demand_model_params,
                    volatility=volatility,
                    nominal_lead_time=nominal_lead_time,
                    std_lead_time=std_lead_time,
                    lead_times_dist=lead_times_dist,
                    review_period=review_period,
                    alpha=alpha,
                    backorders=backorders,
                    )

                    alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                    alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                    beta = np.append(beta, np.array([fill_rates]), axis=0)
                    inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                    
#                     aa = round(alpha, 2)
#                     if aa == 0.80 or aa == 0.85 or aa == 0.90 or aa == 0.95 or aa == 0.99:
#                         print(aa)
#                         print(alpha)
#                         print(round(period_service_levels.mean(),3))
#                         print(round(cycle_service_levels.mean(),3))
#                         print(round(fill_rates.mean(),3))

                min_alpha_p = alpha_p.min(axis=1)
                min_alpha_csl = alpha_csl.min(axis=1)
                min_beta = beta.min(axis=1)
                max_alpha_p = alpha_p.max(axis=1)
                max_alpha_csl = alpha_csl.max(axis=1)
                max_beta = beta.max(axis=1)
                mean_alpha_p = alpha_p.mean(axis=1)
                mean_alpha_csl = alpha_csl.mean(axis=1)
                mean_beta = beta.mean(axis=1)
                
                if v == 'normal':
                    mean_alpha_csl_n = mean_alpha_csl
                
                mean_inv_dem_ratio = np.append(mean_inv_dem_ratio, np.array([ inv_dem_ratio.mean(axis=1)]), axis=0)
               

                print('Lead Time: ',l)
                print('Volatility: ',v)
                print('Backorders: ',b)                
                plotting_fucn_sl(title)
        if v == 'empirical':
            plotting_fucn_ratio(title)
            plotting_fucn_ratio_CSL(title)
                
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Demand: Binomial
##### Lead Time: Deterministic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for l in lead_time_array:
    for b in backorder_array:
        mean_inv_dem_ratio = np.empty((0,59), float)
        for v in volatility_array:
                alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
                alpha_p = np.empty((0,1000), float)
                alpha_csl = np.empty((0,1000), float)
                beta = np.empty((0,1000), float)
                inv_dem_ratio = np.empty((0,1000), float)
                title = 'Demand: Binomial', 'Backorder: ',b,'Volatility',v,'Deterministic Lead Time: ',l
                for a in alpha_array:
                    demand_model ='zip'
                    nominal_lead_time = np.int64(l)
                    std_lead_time = 0
                    lead_times_dist = np.array([l], dtype=np.int64)
                    volatility = v
                    backorders = b
                    num_simulations = 1000
                    num_timesteps_train = 1000
                    num_timesteps_test = 100
                    demand_rate = 10
                    p_demand = 0.3
#                     demand_mean = 100
#                     demand_sd = 20
                    review_period = np.int64(4)
                    alpha = a
                    num_train = 365
                    demand_model_params = {"demand_rate": 10, "p_demand": 0.3}

                    period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                    num_simulations=num_simulations,
                    num_timesteps_train=num_timesteps_train,
                    num_timesteps_test=num_timesteps_test,
                    demand_model=demand_model,
                    demand_model_params=demand_model_params,
                    volatility=volatility,
                    nominal_lead_time=nominal_lead_time,
                    std_lead_time=std_lead_time,
                    lead_times_dist=lead_times_dist,
                    review_period=review_period,
                    alpha=alpha,
                    backorders=backorders,
                    )

                    alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                    alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                    beta = np.append(beta, np.array([fill_rates]), axis=0)
                    inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                    
#                     aa = round(alpha, 2)
#                     if aa == 0.80 or aa == 0.85 or aa == 0.90 or aa == 0.95 or aa == 0.99:
#                         print(aa)
#                         print(alpha)
#                         print(round(period_service_levels.mean(),3))
#                         print(round(cycle_service_levels.mean(),3))
#                         print(round(fill_rates.mean(),3))

                min_alpha_p = alpha_p.min(axis=1)
                min_alpha_csl = alpha_csl.min(axis=1)
                min_beta = beta.min(axis=1)
                max_alpha_p = alpha_p.max(axis=1)
                max_alpha_csl = alpha_csl.max(axis=1)
                max_beta = beta.max(axis=1)
                mean_alpha_p = alpha_p.mean(axis=1)
                mean_alpha_csl = alpha_csl.mean(axis=1)
                mean_beta = beta.mean(axis=1)
                
                if v == 'normal':
                    mean_alpha_csl_n = mean_alpha_csl
                
                mean_inv_dem_ratio = np.append(mean_inv_dem_ratio, np.array([ inv_dem_ratio.mean(axis=1)]), axis=0)
               

                print('Lead Time: ',l)
                print('Volatility: ',v)
                print('Backorders: ',b)                
                plotting_fucn_sl(title)
        if v == 'empirical':
            plotting_fucn_ratio(title)
            plotting_fucn_ratio_CSL(title)

                
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Demand: Binomial
##### Lead Time: Stochastic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for l in lead_time_array:
    for b in backorder_array:
        mean_inv_dem_ratio = np.empty((0,59), float)
        for v in volatility_array:
                alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
                alpha_p = np.empty((0,1000), float)
                alpha_csl = np.empty((0,1000), float)
                beta = np.empty((0,1000), float)
                inv_dem_ratio = np.empty((0,1000), float)
                title = 'Demand: Binomial', 'Backorder: ',b,'Volatility',v,'Stochastic Lead Time: ',l
                for a in alpha_array:
                    demand_model ='zip'
                    nominal_lead_time = np.int64(l)
                    std_lead_time = 2
                    lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                    volatility = v
                    backorders = b
                    num_simulations = 1000
                    num_timesteps_train = 1000
                    num_timesteps_test = 100
                    demand_rate = 10
                    p_demand = 0.3
#                     demand_mean = 100
#                     demand_sd = 20
                    review_period = np.int64(4)
                    alpha = a
                    num_train = 365
                    demand_model_params = {"demand_rate": 10, "p_demand": 0.3}

                    period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                    num_simulations=num_simulations,
                    num_timesteps_train=num_timesteps_train,
                    num_timesteps_test=num_timesteps_test,
                    demand_model=demand_model,
                    demand_model_params=demand_model_params,
                    volatility=volatility,
                    nominal_lead_time=nominal_lead_time,
                    std_lead_time=std_lead_time,
                    lead_times_dist=lead_times_dist,
                    review_period=review_period,
                    alpha=alpha,
                    backorders=backorders,
                    )

                    alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                    alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                    beta = np.append(beta, np.array([fill_rates]), axis=0)
                    inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                    
#                     aa = round(alpha, 2)
#                     if aa == 0.80 or aa == 0.85 or aa == 0.90 or aa == 0.95 or aa == 0.99:
#                         print(aa)
#                         print(alpha)
#                         print(round(period_service_levels.mean(),3))
#                         print(round(cycle_service_levels.mean(),3))
#                         print(round(fill_rates.mean(),3))

                min_alpha_p = alpha_p.min(axis=1)
                min_alpha_csl = alpha_csl.min(axis=1)
                min_beta = beta.min(axis=1)
                max_alpha_p = alpha_p.max(axis=1)
                max_alpha_csl = alpha_csl.max(axis=1)
                max_beta = beta.max(axis=1)
                mean_alpha_p = alpha_p.mean(axis=1)
                mean_alpha_csl = alpha_csl.mean(axis=1)
                mean_beta = beta.mean(axis=1)
                
                if v == 'normal':
                    mean_alpha_csl_n = mean_alpha_csl
                    
                mean_inv_dem_ratio = np.append(mean_inv_dem_ratio, np.array([ inv_dem_ratio.mean(axis=1)]), axis=0)
               

                print('Lead Time: ',l)
                print('Volatility: ',v)
                print('Backorders: ',b)                
                plotting_fucn_sl(title)
        if v == 'empirical':
            plotting_fucn_ratio(title)
            plotting_fucn_ratio_CSL(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Lead Time based graph
##### Demand: Normal
##### Lead Time: Deterministic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in lead_time_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Normal', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='normal'
                nominal_lead_time = np.int64(l)
                std_lead_time = 0
                lead_times_dist = np.array([l], dtype=np.int64)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_mean = 100
                demand_sd = 20
                review_period = np.int64(4)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_mean": 100, "demand_sd": 20}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_lt(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Lead Time based graph
##### Demand: Normal
##### Lead Time: Stochastic
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in lead_time_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Normal', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='normal'
                nominal_lead_time = np.int64(l)
                std_lead_time = 2
                lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_mean = 100
                demand_sd = 20
                review_period = np.int64(4)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_mean": 100, "demand_sd": 20}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                
            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_lt(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Lead Time based graph
##### Demand: Poisson
##### Lead Time: Deterministic 
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in lead_time_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Poisson', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='poisson'
                nominal_lead_time = np.int64(l)
                std_lead_time = 0
                lead_times_dist = np.array([l], dtype=np.int64)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_rate = 10
            #                     demand_mean = 100
            #                     demand_sd = 20
                review_period = np.int64(4)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_rate": 10}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_lt(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Lead Time based graph
##### Demand: Poisson
##### Lead Time: Stochastic
##### Review Period: 4 
##### Lead Time Value : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
lead_time_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in lead_time_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Poisson', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='poisson'
                nominal_lead_time = np.int64(l)
                std_lead_time = 2
                lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_rate = 10
            #                     demand_mean = 100
            #                     demand_sd = 20
                review_period = np.int64(4)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_rate": 10}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_lt(title)

print("--- %s seconds ---" % (time.time() - start_time))


Varying Review Periods

In [ ]:
# Plotting function for lead time
def plotting_fucn_rp(title):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    plt.plot(alpha_array, csl_for_lead_time[0,:], label = 'Review Period: 2')
    
    plt.plot(alpha_array, csl_for_lead_time[1,:], label = 'Review Period: 4')
    
    plt.plot(alpha_array, csl_for_lead_time[2,:], label = 'Review Period: 10')

    plt.grid(axis='x', color='0.95')
    
    ax.set(xlabel='Target Service Level', ylabel='Achieved Service Level') 
    ax.set_title(title)
    plt.legend(loc='lower right', prop={'size': 12})
    plt.xlim(0.7 , 0.99)
    plt.show()
#     filename =  str(title) +'.png'
#     plt.savefig(filename, bbox_inches='tight')
    
    return (ax)

In [ ]:
#### Review period based graph
##### Demand: Normal
##### Lead Time: Deterministic 
##### Lead TIme: 4 
##### Review Period : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
review_period_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in review_period_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Normal', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='normal'
                nominal_lead_time = np.int64(4)
                std_lead_time = 0
                lead_times_dist = np.array([4], dtype=np.int64)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_mean = 100
                demand_sd = 20
                review_period = np.int64(l)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_mean": 100, "demand_sd": 20}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_rp(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Review period based graph
##### Demand: Normal
##### Lead Time: Stochatic 
##### Lead Time: 4 
##### Review Period : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
review_period_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in review_period_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Normal', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='normal'
                nominal_lead_time = np.int64(4)
                std_lead_time = 2
                lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_mean = 100
                demand_sd = 20
                review_period = np.int64(l)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_mean": 100, "demand_sd": 20}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                
            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_rp(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Review period based graph
##### Demand: Poisson
##### Lead Time: Deterministic 
##### Lead TIme: 4 
##### Review Period : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
review_period_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in review_period_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Poisson', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='poisson'
                nominal_lead_time = np.int64(4)
                std_lead_time = 0
                lead_times_dist = np.array([4], dtype=np.int64)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_rate = 10
            #                     demand_mean = 100
            #                     demand_sd = 20
                review_period = np.int64(l)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_rate": 10}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_rp(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Review period based graph
##### Demand: Poisson
##### Lead Time: Stochastic 
##### Lead Time: 4 
##### Review Period : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
review_period_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in review_period_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Poisson', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='poisson'
                nominal_lead_time = np.int64(4)
                std_lead_time = 2
                lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_rate = 10
            #                     demand_mean = 100
            #                     demand_sd = 20
                review_period = np.int64(l)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_rate": 10}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_rp(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Review period based graph
##### Demand: Combined
##### Lead Time: Deterministic 
##### Lead Time: 4 
##### Review Period : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
review_period_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in review_period_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Combined Binomial and Poisson', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='zip'
                nominal_lead_time = np.int64(4)
                std_lead_time = 0
                lead_times_dist = np.array([4], dtype=np.int64)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                demand_rate = 10
                p_demand = 0.3
            #                     demand_mean = 100
            #                     demand_sd = 20
                review_period = np.int64(l)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_rate": 10, "p_demand": 0.3}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)
                

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_rp(title)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#### Review period based graph
##### Demand: Combined
##### Lead Time: Stochastic 
##### Lead Time: 4 
##### Review Period : 2, 4, 10
##### Volatility : Normal, Empirical
##### Backorder : False, True
start_time = time.time()
review_period_array = [2,4,10]
volatility_array = ['normal','empirical']
backorder_array = [False,True]
for b in backorder_array:
    for v in volatility_array:
        csl_for_lead_time = np.empty((0,59), float)
        for l in review_period_array:
            alpha_array = np.arange(start=0.7, stop=0.99, step=0.005)
            alpha_p = np.empty((0,1000), float)
            alpha_csl = np.empty((0,1000), float)
            beta = np.empty((0,1000), float)
            inv_dem_ratio = np.empty((0,1000), float)
            title = 'Demand: Poisson', 'Backorder: ',b,'Volatility',v
            for a in alpha_array:
                demand_model ='zip'
                nominal_lead_time = np.int64(4)
                std_lead_time = 2
                lead_times_dist = np.random.randint(nominal_lead_time,nominal_lead_time+std_lead_time+1,1000)
                volatility = v
                backorders = b
                num_simulations = 1000
                num_timesteps_train = 1000
                num_timesteps_test = 100
                p_demand = 0.3
            #                     demand_mean = 100
            #                     demand_sd = 20
                review_period = np.int64(l)
                alpha = a
                num_train = 365
                demand_model_params = {"demand_rate": 10, "p_demand: 0.3"}

                period_service_levels, cycle_service_levels, fill_rates,inventory_demand_ratio = run_study(
                num_simulations=num_simulations,
                num_timesteps_train=num_timesteps_train,
                num_timesteps_test=num_timesteps_test,
                demand_model=demand_model,
                demand_model_params=demand_model_params,
                volatility=volatility,
                nominal_lead_time=nominal_lead_time,
                std_lead_time=std_lead_time,
                lead_times_dist=lead_times_dist,
                review_period=review_period,
                alpha=alpha,
                backorders=backorders,
                )

                alpha_p = np.append(alpha_p, np.array([period_service_levels]), axis=0)
                alpha_csl = np.append(alpha_csl, np.array([cycle_service_levels]), axis=0)
                beta = np.append(beta, np.array([fill_rates]), axis=0)
#                 inv_dem_ratio = np.append(inv_dem_ratio, np.array([inventory_demand_ratio]), axis=0)

            min_alpha_p = alpha_p.min(axis=1)
            min_alpha_csl = alpha_csl.min(axis=1)
            min_beta = beta.min(axis=1)
            max_alpha_p = alpha_p.max(axis=1)
            max_alpha_csl = alpha_csl.max(axis=1)
            max_beta = beta.max(axis=1)
            mean_alpha_p = alpha_p.mean(axis=1)
            mean_alpha_csl = alpha_csl.mean(axis=1)
            mean_beta = beta.mean(axis=1)
            csl_for_lead_time = np.append(csl_for_lead_time, np.array([mean_alpha_csl]), axis=0)

        plotting_fucn_rp(title)

print("--- %s seconds ---" % (time.time() - start_time))
